# Load PERS exchange rate data into silver layer


In [1]:
-- you will notice there's an issue with the currency pair in this data
SELECT * FROM PERS.bronze_fxdata WHERE to_date(cast(unix_timestamp(`YYYY/MM/DD`, 'yyyy/MM/dd') as timestamp)) = '2024-01-02' LIMIT 10;
-- basically, base currency and quote currency from the PERS FX dataset is flipped in the bronze data

StatementMeta(, a1fde997-f5f7-4b26-8861-657b1401229f, 2, Finished, Available)

<Spark SQL result set with 10 rows and 5 fields>

###### Create temporary views with data transformation logic
Notice that base currency and quote currency is flipped in the view to accommodate for data issues

In [2]:
CREATE VIEW IF NOT EXISTS vwcurrency AS
SELECT 1 as col;

-- create temp view vwcurrency.  this will be used to populate currency dimension
ALTER VIEW vwcurrency AS
WITH currencycte AS (
SELECT DISTINCT split(ISO4217, '/')[0] AS `currency_id`
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
UNION 
SELECT DISTINCT split(ISO4217, '/')[1] AS `currency_id`
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
)
SELECT DISTINCT `currency_id` FROM currencycte;

CREATE VIEW IF NOT EXISTS vwexchangerate AS
SELECT 1 as col;

-- create temp view vwexchangerate.  this will be used to populate exchange rate fact table
ALTER VIEW vwexchangerate AS
SELECT     
    split(ISO4217, '/')[1] AS `base_currency_id`, 
    split(ISO4217, '/')[0] AS `quote_currency_id`,
    to_date(cast(unix_timestamp(`YYYY/MM/DD`, 'yyyy/MM/dd') as timestamp)) AS `exchange_date`,
    Data AS `exchange_rate` 
FROM PERS.bronze_fxdata
WHERE ISO4217 LIKE '%/%'
AND Data IS NOT NULL
AND `YYYY/MM/DD` IS NOT NULL;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

###### Create silver dimension and fact tables if not exists

In [3]:
-- create currency table
CREATE TABLE IF NOT EXISTS PERS.silver_currency
(
    currency_id string,
    currency_name string,
    create_dt timestamp,
    modify_dt timestamp
) USING delta;

-- create exchange_rate table
CREATE TABLE IF NOT EXISTS PERS.silver_exchange_rate
(
    base_currency_id string,
    quote_currency_id string,
    exchange_date date,
    exchange_rate double,
    create_dt timestamp,
    modify_dt timestamp
) USING delta; 

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

###### load currency dimension table

In [4]:
MERGE INTO PERS.silver_currency TGT
USING vwcurrency SRC
ON TGT.currency_id = SRC.currency_id
WHEN NOT MATCHED THEN
INSERT (currency_id,currency_name,create_dt,modify_dt)
VALUES (SRC.currency_id,SRC.currency_id,CURRENT_TIMESTAMP,CURRENT_TIMESTAMP);

StatementMeta(, a1fde997-f5f7-4b26-8861-657b1401229f, 9, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

###### Load exchange rate fact table

In [5]:
MERGE INTO PERS.silver_exchange_rate TGT
USING vwexchangerate SRC
ON (TGT.base_currency_id = SRC.base_currency_id AND TGT.quote_currency_id = SRC.quote_currency_id AND TGT.exchange_date = CAST(SRC.exchange_date AS DATE))
WHEN MATCHED AND TGT.exchange_rate <> SRC.exchange_rate THEN
    UPDATE SET TGT.exchange_rate = SRC.exchange_rate,
    TGT.modify_dt = CURRENT_TIMESTAMP
WHEN NOT MATCHED THEN
    INSERT (base_currency_id, quote_currency_id, exchange_date, exchange_rate, create_dt, modify_dt) 
    VALUES (SRC.base_currency_id, SRC.quote_currency_id, SRC.exchange_date, SRC.exchange_rate, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);

StatementMeta(, a1fde997-f5f7-4b26-8861-657b1401229f, 10, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>